In [ ]:
%%bash
pip install gym-super-mario-bros==7.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.7/77.7 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.1 MB/s eta 0:00:00
  Created wheel for nes-py: filename=nes_py-8.2.1-cp310-cp310-linux_x86_64.whl size=535719 sha256=f985c73427fb7b67264bb4ffccdb57625b71a9bcc67433db41e126159c1a6305
  Stored in directory: /root/.cache/pip/wheels/34/a7/d5/9aa14b15df740a53d41f702e4c795731b6c4da7925deb8476c
Successfully built nes-py


In [ ]:
%%bash
pip install tensordict
pip install torchrl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 986.5/986.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 14.5 MB/s eta 0:00:00


In [ ]:
!pip install pyglet[glu]

In [ ]:
%%bash
pip install stable_baselines3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 17.4 MB/s eta 0:00:00


In [ ]:
!pip install -Uqq ipdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.6 MB/s eta 0:00:00


In [ ]:
import torch
from torch import nn
from torchvision import transforms as T
from PIL import Image
import numpy as np
from pathlib import Path
from collections import deque
import random, datetime, os, copy
import gc
import gym
from gym.spaces import Box
from gym.wrappers import FrameStack
from tqdm import tqdm
from collections import deque
import gc
from torchrl.data import TensorDictReplayBuffer, LazyMemmapStorage
import time
import matplotlib.pyplot as plt
from tensordict import TensorDict
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.policies import obs_as_tensor
from stable_baselines3.common.evaluation import evaluate_policy
import imageio
import matplotlib.pyplot as plt
from matplotlib import colors
from gym import spaces

# NES Emulator for OpenAI Gym
from nes_py.wrappers import JoypadSpace

# Super Mario environment for OpenAI Gym
import gym_super_mario_bros
import ipdb

In [ ]:
%pdb on

Automatic pdb calling has been turned ON


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
class SMB():
    '''
    Wrapper function containing the processed environment and the loaded model
    '''
    def __init__(self, env, model):
        self.env = env
        self.model = model

    def play(self, episodes=5, deterministic=False, render=True, return_eval=False):
        for episode in range(1, episodes+1):
            states = self.env.reset()
            done = False
            score = 0

            if render == True:
                while not done:
                    self.env.render()
                    action, _ = self.model.predict(states, deterministic=deterministic)
                    states, reward, done, info = self.env.step(action)
                    score += reward
                    time.sleep(0.01)
                print('Episode:{} Score:{}'.format(episode, score))
            else:
                while not done:
                    action, _ = self.model.predict(states, deterministic=deterministic)
                    states, reward, done, info = self.env.step(action)
                    score += reward
        if return_eval == True:
            return score, info
        else:
            return

    def evaluate(self, episodes=20, deterministic=False):
        '''
        returns rewards, steps (both have length [episodes])
        '''
        rewards, steps = evaluate_policy(self.model, self.env, n_eval_episodes=episodes,
                                 deterministic=deterministic, render=False,
                                 return_episode_rewards=True)
        return rewards, steps




    def predict_proba(self, state):
        '''
        Predict the probability of each action given a state
        https://stackoverflow.com/questions/66428307/how-to-get-action-propability-in-stable-baselines-3/70012691#70012691?newreg=bd5479b970664069b359903e0151b4a1
        '''
        model = self.model
        obs = obs_as_tensor(state, model.policy.device)
        dis = model.policy.get_distribution(obs)
        probs = dis.distribution.probs
        probs_np = probs.detach().numpy()
        return probs_np

    #############
    # functions for making plots & videos

    def make_video_frames(self, deterministic=False):
        '''
        For each step, plot obs & rendered screen in one figure for making videoes
        '''
        state = self.env.reset()
        done = False
        score = [0]
        #self._make_combined_plot2(state, score, prob_actions)
        #self._make_combined_plot(state, score)


        while not done:
        #for i in range(1):
            prob_actions = self.predict_proba(state)
            action, _ = self.model.predict(state, deterministic=deterministic)
            state, reward, done, info = self.env.step(action)
            score += reward
            self._make_combined_plot2(state, score, prob_actions)
            #self._make_combined_plot(state, score)


    def _make_combined_plot2(self, state, score, prob_actions):
        '''
        Originally made for n_stack = 4 & n_skip = 4, SIMPLE_MOVEMENT
        '''
        # get rendered screen
        im_render = self.env.render(mode="rgb_array")

        n_stack = state.shape[-1]
        cmap = colors.ListedColormap(['red', 'skyblue', 'brown', 'blue'])
        bounds = [-1.5, -0.5, 0.5, 1.5, 2.5]
        norm = colors.BoundaryNorm(bounds, cmap.N)

        #obs_loc = [[0, 1], [0, 2], [1, 1], [1, 2]]
        obs_loc = [[0, 1], [1, 1], [2, 1], [3, 1]]
        obs_text = ['t (current frame)', 't-4', 't-8', 't-12']
        action_list = ['NOOP', 'right', 'right+A', 'right+B', 'right+A+B', 'A', 'left']


        ##########
        fig = plt.figure(dpi=100, figsize=(6, 6), constrained_layout=False, tight_layout=True)
        gs = fig.add_gridspec(4, 2, width_ratios=[3, 1])

        # individual obs frames
        for n in range(n_stack):
            ax = fig.add_subplot(gs[obs_loc[n][0], obs_loc[n][1]])
            im = ax.imshow(state[0,:,:,n], cmap=cmap, norm=norm)
            ax.set_axis_off()
            ax.text(-0.5, 14.5, obs_text[n])

        # prob_actions
        ax = fig.add_subplot(gs[3, 0])
        ax.bar(action_list, prob_actions[0])
        plt.xticks(rotation=45)
        ax.set_ylim(0, 1.05)

        # rendered screen
        ax = fig.add_subplot(gs[0:3, 0])
        im = ax.imshow(im_render)
        ax.set_axis_off()
        ax.text(0, -5, 'score: '+str(int(score[0])))

        plt.show()


    def _make_combined_plot(self, state, score):
        # get rendered screen
        im_render = self.env.render(mode="rgb_array")
        n_stack = state.shape[-1]

        cmap = colors.ListedColormap(['red', 'skyblue', 'brown', 'blue'])
        bounds = [-1.5, -0.5, 0.5, 1.5, 2.5]
        norm = colors.BoundaryNorm(bounds, cmap.N)

        #obs_text = ['t (current frame)', 't-4', 't-8', 't-12']

        fig = plt.figure(dpi=100, figsize=(5.5, 4), constrained_layout=False, tight_layout=True)
        gs = fig.add_gridspec(4, 2, width_ratios=[4, 1])

        # individual obs frames
        for n in range(n_stack):
            ax = fig.add_subplot(gs[n, 1])
            im = ax.imshow(state[0,:,:,n], cmap=cmap, norm=norm)
            ax.set_axis_off()

        # rendered screen
        ax = fig.add_subplot(gs[:, 0])
        im = ax.imshow(im_render)
        ax.set_axis_off()
        ax.text(0, -5, 'score: '+str(int(score[0])))

        plt.show()

    def make_animation(self, deterministic=True, filename='gym_animation.gif', RETURN_FRAMES=False):
        '''
        Make an animation of the rendered screen
        '''
        # run policy
        frames = []
        states = self.env.reset()
        done = False

        while not done:
            #frames.append(self.env.render(mode="rgb_array"))
            im = self.env.render(mode="rgb_array")
            frames.append(im.copy())
            action, _ = self.model.predict(states, deterministic=deterministic)
            states, reward, done, info = self.env.step(action)

        if RETURN_FRAMES == False:
            # make animation
            imageio.mimsave(filename, frames, fps=50)
        else: # make animation manually in case Mario gets stuck in the level and drags the animation for too long
            return frames

In [ ]:
class smb_grid:

    def __init__(self, env):
        self.ram = env.unwrapped.ram
        self.screen_size_x = 16    # rendered screen size
        self.screen_size_y = 13

        self.mario_level_x = self.ram[0x6d]*256 + self.ram[0x86]
        self.mario_x = self.ram[0x3ad]  # mario's position on the rendered screen
        self.mario_y = self.ram[0x3b8] + 16 # top edge of (big) mario

        self.x_start = self.mario_level_x - self.mario_x # left edge pixel of the rendered screen in level
        self.rendered_screen = self.get_rendered_screen()


    ########
    # get background tile grid

    def tile_loc_to_ram_address(self, x, y):
        '''
        convert (x, y) in Current tile (32x13, stored as 16x26 in ram) to ram address
        x: 0 to 31
        y: 0 to 12
        '''
        page = x // 16
        x_loc = x%16
        y_loc = page*13 + y

        address = 0x500 + x_loc + y_loc*16

        return address

    def get_rendered_screen(self):
        '''
        Get the rendered screen (16 x 13) from ram
        empty: 0
        tile: 1
        enemy: -1
        mario: 2
        '''

        # Get background tiles

        rendered_screen = np.zeros((self.screen_size_y, self.screen_size_x))
        screen_start = int(np.rint(self.x_start / 16))

        for i in range(self.screen_size_x):
            for j in range(self.screen_size_y):
                x_loc = (screen_start + i) % (self.screen_size_x * 2)
                y_loc = j
                address = self.tile_loc_to_ram_address(x_loc, y_loc)
                #bg_screen2[j, i] = env.unwrapped.ram[address]

                # Convert all types of tile to 1
                if self.ram[address] != 0:
                    rendered_screen[j, i] = 1

        # Add mario
        x_loc = (self.mario_x + 8) // 16
        y_loc = (self.mario_y - 32) // 16 # top 2 rows in the rendered screen aren't stored in ram
        if x_loc < 16 and y_loc < 13:
            rendered_screen[y_loc, x_loc] = 2

        # Add enemies
        for i in range(5):
            # check if the enemy is drawn
            if self.ram[0xF + i] == 1:
                enemy_x = self.ram[0x6e + i]*256 + self.ram[0x87 + i] - self.x_start
                enemy_y = self.ram[0xcf + i]
                x_loc = (enemy_x + 8) // 16
                y_loc = (enemy_y + 8 - 32) // 16

                # check if the enemy is inside the rendered screen
                # 8/6/22 fixed bug where enemy with x_loc < 0 still got added to rendered_screen; doesn't seem to affect trained models' performance
                # if x_loc < 16 and y_loc < 13:
                if 0 <= x_loc < 16 and 0 <= y_loc < 13:
                    rendered_screen[y_loc, x_loc] = -1

        return rendered_screen

In [ ]:
class SMBRamWrapper(gym.ObservationWrapper):
    def __init__(self, env, crop_dim=[0, 16, 0, 13], n_stack=4, n_skip=2):
        '''
        crop_dim: [x0, x1, y0, y1]
        obs shape = (height, width, n_stack), n_stack=0 is the most recent frame
        n_skip: e.g. n_stack=4, n_skip=2, use frames [0, 2, 4, 6]
        '''
        gym.Wrapper.__init__(self, env)
        self.crop_dim = crop_dim
        self.n_stack = n_stack
        self.n_skip = n_skip
        # Modified from stable_baselines3.common.atari_wrappers.WarpFrame()
        # https://stable-baselines3.readthedocs.io/en/master/_modules/stable_baselines3/common/atari_wrappers.html#AtariWrapper
        self.width = crop_dim[1] - crop_dim[0]
        self.height = crop_dim[3] - crop_dim[2]
        self.observation_space = spaces.Box(
            low=-1, high=2, shape=(self.height, self.width, self.n_stack), dtype=int
        )

        self.frame_stack = np.zeros((self.height, self.width, (self.n_stack-1)*self.n_skip+1))
        #self.INDEX_SKIP = 1

    def observation(self, obs):
        grid = smb_grid(self.env)
        frame = grid.rendered_screen # 2d array
        frame = self.crop_obs(frame)

        self.frame_stack[:,:,1:] = self.frame_stack[:,:,:-1] # shift frame_stack by 1
        self.frame_stack[:,:,0] = frame # add current frame to stack
        obs = self.frame_stack[:,:,::self.n_skip]
        return obs

    def reset(self):
        obs = self.env.reset()
        self.frame_stack = np.zeros((self.height, self.width, (self.n_stack-1)*self.n_skip+1))
        grid = smb_grid(self.env)
        frame = grid.rendered_screen # 2d array
        frame = self.crop_obs(frame)
        for i in range(self.frame_stack.shape[-1]):
            self.frame_stack[:,:,i] = frame
        obs = self.frame_stack[:,:,::self.n_skip]
        return obs

    def crop_obs(self, im):
        '''
        Crop observed frame image to reduce input size
        Returns cropped_frame = original_frame[y0:y1, x0:x1]
        '''
        [x0, x1, y0, y1] = self.crop_dim
        im_crop = im[y0:y1, x0:x1]
        return im_crop


In [ ]:
class MarioNet(nn.Module):

    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.online = nn.Sequential(

            nn.Linear(input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, output_dim),
        )

        self.target = copy.deepcopy(self.online)

        # Q_target parameters are frozen.
        for p in self.target.parameters():
            p.requires_grad = False

    def forward(self, input, model):
        if model == "online":
            return self.online(input)
        elif model == "target":
            return self.target(input)

In [ ]:
class World:
    def __init__(self, env):
        self.env = env
        self.mario_pos = 0
        self.enemy_pos = []
        self.episodes = 10000
        self.episolon = 1
        self.episilon_rate = 1/self.episodes
        self.Q_table = {}
        self.store_state = []
        self.rng = np.random.default_rng()
        self.total_reward = []

    def update_epsilon(self, episode):
        self.episolon = max(0.1, self.episolon - episode * self.episilon_rate)

    def find_mario(self, state):
        mask = (state == 2)
        if len(np.argwhere(mask)) == 0:
            return mask
        return np.argwhere(mask)[0]

    def find_enemy(self, state):
        mask = (state == -1)
        return np.argwhere(mask)


    def check_ground(self, state):
        mario_pos = self.find_mario(state)
        if mario_pos[0] + 1 == 13:
            return False
        return (state[mario_pos[0] + 1, mario_pos[1]] == 1)

    def obstacle_ahead(self, state):
        mario_pos = self.find_mario(state)
        return state[mario_pos[0] - 1: mario_pos[0] + 2, mario_pos[1] + 1]

    def obstacle_near(self, state):
        mario_pos = self.find_mario(state)
        return state[mario_pos[0] - 1: mario_pos[0] + 2, mario_pos[1] + 2]

    def can_jump(self, state):
        mario_pos = self.find_mario(state)
        return (state[mario_pos[0] - 1, mario_pos[1]] != 1) and (state[mario_pos[0] - 2,mario_pos[1]] != 1)

    def enemy_near(self, state):
        mario_pos = self.find_mario(state)
        enemies = self.find_enemy(state)
        max_dist = 0
        if len(enemies) == 0:
            return False
        for enemy in enemies:
            if enemy[1] - mario_pos[1] < 0: continue
            dist = max(abs(enemy[0] - mario_pos[0]), enemy[1] - mario_pos[1])
            max_dist = max(max_dist, dist)
        return max_dist <= 2

    def enemy_bellow(self, state):
        mario_pos = self.find_mario(state)
        if state[mario_pos[0] + 1, mario_pos[1]] == -1:
            return True
        return False

    def enemy_mid(self, state):
        mario_pos = self.find_mario(state)
        enemies = self.find_enemy(state)
        max_dist = 0
        if len(enemies) == 0:
            return False
        for enemy in enemies:
            if enemy[1] - mario_pos[1] < 0: continue
            dist = max(abs(enemy[0] - mario_pos[0]), enemy[1] - mario_pos[1])
            max_dist = max(max_dist, dist)
        return max_dist <= 3

    def enemy_far(self, state):
        mario_pos = self.find_mario(state)
        enemies = self.find_enemy(state)
        max_dist = 0
        if len(enemies) == 0:
            return False
        for enemy in enemies:
            if enemy[1] - mario_pos[1] < 0: continue
            dist = max(abs(enemy[0] - mario_pos[0]), enemy[1] - mario_pos[1])
            max_dist = max(max_dist, dist)
        return max_dist <= 4

    def colision(self, state):
        mario_pos = self.find_mario(state)
        enemies = self.find_enemy(state)
        colision = False
        for enemy in enemies:
            if (mario_pos[0] - enemy[0] == 1) or (enemy[1] - mario_pos[1] == 1):
                colision = True

        return colision


    def get_state_action_tuple(self, state):
        ground = self.check_ground(state)
        jump = self.can_jump(state)
        collision = self.colision(state)
        near = self.enemy_near(state)
        mid = self.enemy_mid(state)
        far = self.enemy_far(state)
        obstacle = self.obstacle_ahead(state)
        enemy_bellow = self.enemy_bellow(state)
        obstacle_near = self.obstacle_near(state)
        return tuple([ground, jump, collision, near, mid, far, obstacle[0], obstacle[1], obstacle[2], enemy_bellow,
                       obstacle_near[0], obstacle_near[1], obstacle_near[2]])

    def get_states_action_tuple(self, states):
        tp = tuple([])
        for i in range(states.shape[2]):
            tp += self.get_state_action_tuple(states[:,:,i])
        return np.asarray(tp)


    def adjust_reward(self, state, reward, action):
        enemies = self.find_enemy(state)
        mario_pos = self.find_mario(state)
        if action == 0 or mario_pos.shape[0] != 3:
            return reward
        if state[mario_pos[0] + 1, mario_pos[1] + 1, 0] == 0: #não penalizar por pular poço
            return reward
        if state[mario_pos[0], mario_pos[1] + 1, 0] == 1 or state[mario_pos[0], mario_pos[1] + 2, 0] == 1: #não penalisar por pular obstáculo
            return reward
        if len(enemies) == 0:
            return - 1
        for enemy in enemies:
            if (enemy[1] - mario_pos[1]) > 2: #penalisar o mario por pular de forma desnecessária
                return -1

        return reward



In [ ]:

class Mario:
    def __init__(self, state_dim, action_dim, save_dir):
        self.state_dim = state_dim
        self.action_dim = action_dim
        self.save_dir = save_dir

        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        # Mario's DNN to predict the most optimal action - we implement this in the Learn section
        self.net = MarioNet(self.state_dim, self.action_dim).float()
        self.net = self.net.to(device=self.device)

        self.exploration_rate = 1
        self.exploration_rate_decay = 0.99999975
        self.exploration_rate_min = 0.1
        self.curr_step = 0

        self.save_every = 5e5  # no. of experiences between saving Mario Net
        self.memory = TensorDictReplayBuffer(storage=LazyMemmapStorage(100000, device=torch.device("cpu")))
        self.batch_size = 32
        self.gamma = 0.9
        self.optimizer = torch.optim.Adam(self.net.parameters(), lr=0.00025)
        self.loss_fn = torch.nn.SmoothL1Loss()
        self.burnin = 1e4  # min. experiences before training
        self.learn_every = 3  # no. of experiences between updates to Q_online
        self.sync_every = 1e4  # no. of experiences between Q_target & Q_online sync

    def act(self, state):

        # EXPLORE
        if np.random.rand() < self.exploration_rate:
            action_idx = np.random.randint(self.action_dim)

        # EXPLOIT
        else:
            state = state[0].__array__() if isinstance(state, tuple) else state.__array__()
            state = torch.tensor(state, device=self.device).unsqueeze(0)
            state = state.to(torch.float)
            action_values = self.net(state, model="online")
            action_idx = torch.argmax(action_values, axis=1).item()

        # decrease exploration_rate
        self.exploration_rate *= self.exploration_rate_decay
        self.exploration_rate = max(self.exploration_rate_min, self.exploration_rate)

        # increment step
        self.curr_step += 1
        return action_idx


    def cache(self, state, next_state, action, reward, done):
        def first_if_tuple(x):
            return x[0] if isinstance(x, tuple) else x
        state = first_if_tuple(state).__array__()
        next_state = first_if_tuple(next_state).__array__()

        state = torch.tensor(state)
        next_state = torch.tensor(next_state)
        action = torch.tensor([action])
        reward = torch.tensor([reward])
        done = torch.tensor([done])

        self.memory.add(TensorDict({"state": state, "next_state": next_state, "action": action, "reward": reward, "done": done}, batch_size=[]))

    def recall(self):
        batch = self.memory.sample(self.batch_size).to(self.device)
        state, next_state, action, reward, done = (batch.get(key) for key in ("state", "next_state", "action", "reward", "done"))
        return state, next_state, action.squeeze(), reward.squeeze(), done.squeeze()

    def td_estimate(self, state, action):
        state = state.to(self.device)
        state = state.to(torch.float)
        action = action.to(self.device)
        current_Q = self.net(state, model="online")[
            np.arange(0, self.batch_size), action
        ]  # Q_online(s,a)
        return current_Q

    @torch.no_grad()
    def td_target(self, reward, next_state, done):
        next_state = next_state.to(torch.float)
        next_state_Q = self.net(next_state, model="online")
        best_action = torch.argmax(next_state_Q, axis=1)
        next_Q = self.net(next_state, model="target")[
            np.arange(0, self.batch_size), best_action
        ]
        return (reward + (1 - done.float()) * self.gamma * next_Q).float()

    def update_Q_online(self, td_estimate, td_target):
        loss = self.loss_fn(td_estimate, td_target)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        return loss.item()

    def sync_Q_target(self):
        self.net.target.load_state_dict(self.net.online.state_dict())

    def save(self):
        save_path = (
            self.save_dir / f"mario_net_{int(self.curr_step // self.save_every)}.chkpt"
        )
        torch.save(
            dict(model=self.net.state_dict(), exploration_rate=self.exploration_rate),
            save_path,
        )
        print(f"MarioNet saved to {save_path} at step {self.curr_step}")


    def learn(self):
        if self.curr_step % self.sync_every == 0:
            self.sync_Q_target()

        if self.curr_step % self.save_every == 0:
            self.save()

        if self.curr_step < self.burnin:
            return None, None

        if self.curr_step % self.learn_every != 0:
            return None, None
        # Sample from memory
        state, next_state, action, reward, done = self.recall()

        # Get TD Estimate
        td_est = self.td_estimate(state, action)

        # Get TD Target
        td_tgt = self.td_target(reward, next_state, done)

        # Backpropagate loss through Q_online
        loss = self.update_Q_online(td_est, td_tgt)


        return (td_est.mean().item(), loss)

In [ ]:
class Game:

    def __init__(self):
        self.x0 = 0
        self.x1 = 16
        self.y0 = 0
        self.y1 = 13
        self.n_stack = 4
        self.n_skip = 4
        self.alfa = 0.5
        self.gamma = 0.7
        self.Episodes = 2000
        env = gym_super_mario_bros.make('SuperMarioBros-1-1-v1')
        env = JoypadSpace(env, [["right"], ["right", "A"]])
        self.env_wrap = SMBRamWrapper(env, [self.x0, self.x1, self.y0, self.y1], n_stack=self.n_stack, n_skip=self.n_skip)
        self.world = World(env)
        save_dir = Path("checkpoints")
        save_dir.mkdir(parents=True, exist_ok=True)
        self.mario = Mario(52, 2, save_dir)


    def update_q_table(self, target, states, action):
        if self.world.check_dict(self.world.get_states_action_tuple(states, action)):
            old_value = self.world.Q_table[self.world.get_states_action_tuple(states, action)]
            self.world.Q_table[self.world.get_states_action_tuple(states, action)] = (1 - self.alfa) * old_value + target
        else:
            self.world.store_state.append(self.world.get_states_action_tuple(states, action))
            self.world.Q_table.update({self.world.get_states_action_tuple(states, action): target})

    def adjust_reward(self, state, reward, action, repeat_pos, info, done):
        reward = self.world.adjust_reward(state, reward, action)
        if repeat_pos[info['x_pos']] > 300:
            reward -= 15
            done = True
        return done

    def check_mario(self, states):
        for i in range(states.shape[2]):
            if len(np.argwhere(self.world.find_mario(states[:,:,i]))) == 0 or self.world.find_mario(states[:,:,i]).shape[0] > 3:
                return False
        return True

    def play(self):
        done = False
        states = self.env_wrap.reset()
        total_reward = 0
        repeat_pos = np.zeros(4000)
        max_pos = 0
        while(not done):
            if self.check_mario(states) is False:
                break
            state = self.world.get_states_action_tuple(states)
            action = self.mario.act(state)
            new_states, reward, done, info = self.env_wrap.step(action)
            if self.check_mario(new_states) is False:
                break
            done = self.adjust_reward(states[:,:,0],reward,action,repeat_pos,info, done)
            new_state = self.world.get_states_action_tuple(new_states)
            self.mario.cache(state, new_state, action, reward, done)
            q, loss = self.mario.learn()
            states = new_states
            repeat_pos[info['x_pos']] += 1
            total_reward += reward
            max_pos = info['x_pos']
            if done or info["flag_get"]:
                break
        return total_reward, max_pos

    def run(self):
        with tqdm(total= self.Episodes) as pbar:
            for episode in range(self.Episodes):
                total_reward, max_pos = self.play()
                self.world.update_epsilon(episode)
                self.world.total_reward.append(total_reward)
                if episode % 100 == 0:
                    print(f'Epoca {episode + 1}: {total_reward}')
                    print(max_pos)
                pbar.update(1)



In [ ]:
game = Game()
game.run()

/usr/local/lib/python3.10/dist-packages/gym/envs/registration.py:593: UserWarning: WARN: The environment SuperMarioBros-1-1-v1 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
  0%|          | 0/2000 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:227: DeprecationWarning: WARN: Core environment is written in old step API which

Epoca 1: 231.0
296


  5%|▌         | 101/2000 [05:40<1:43:52,  3.28s/it]

Epoca 101: 231.0
296


 10%|█         | 201/2000 [11:30<1:18:32,  2.62s/it]

...

 95%|█████████▍| 1894/2000 [1:54:14<08:05,  4.58s/it]

MarioNet saved to checkpoints/mario_net_2.chkpt at step 1000000


 95%|█████████▌| 1901/2000 [1:54:45<05:55,  3.59s/it]

Epoca 1901: 231.0
296


100%|██████████| 2000/2000 [2:01:02<00:00,  3.63s/it]


In [ ]:
%pdb off

Automatic pdb calling has been turned OFF
